In [76]:
import pandas as pd
import os

In [77]:
Sales = pd.read_csv("files/salesClean.csv")
Flights = pd.read_csv("files/limpios2023.csv")

In [78]:
Sales

,Flight_ID,ProductType,ProductName,Quantity,TotalSales
0,a05290288259526edd3601160b10e1de,Botanas,Carne Seca Habanero,1,55.0
1,08f6f97437df8db101b050f1110be656,Licores,Jw Red Label,2,240.0
2,4e09c949826a77207868412baeff6d30,Licores,Jack And Coke,8,576.0
3,c3e1568fe46c68d3174681d322d412b3,Licores,Jw Red Label,2,240.0
4,34ae909bf3699372fcd66f02af0bdd54,Licores,Ron Bacardi,2,240.0
...,...,...,...,...,...
2146675,638cf1904f936c50aea4a6f123ee07cf,Licores,Vino Tinto Sangre De Toro,1,155.0
2146676,04c5a19b8382a888d5c4c7b4932e783e,Licores,Vino Tinto Sangre De Toro,1,155.0
2146677,0291f6b201d2b24717a8ed7ca04f2d90,Licores,Vino Tinto Sangre De Toro,1,155.0
2146678,a10e3ac614fd434d6312897e07727c08,Licores,Vino Tinto Sangre De Toro,1,155.0


In [79]:
join = Flights.merge(Sales, on="Flight_ID")
print(len(join))
a = join['Flight_ID'].unique()
f_unique = Flights['Flight_ID'].unique()


2135157


In [80]:
len(a),len(Flights),len(b)

(103863, 121724, 119151)

In [89]:
duplicate_flights = Flights[Flights['Flight_ID'].duplicated()]['Flight_ID']
duplicate_list = duplicate_flights.tolist()
print(len(duplicate_list))
# Filtrar DataFrame excluyendo filas con IDs en la lista
noDuplicateFlights = Flights[~Flights['Flight_ID'].isin(duplicate_list)]
len(noDuplicateFlights)
prt

2573


116738

In [82]:
join = Flights.merge(Sales, on="Flight_ID")
len(join)

2135157

In [83]:
# Identify duplicate flight IDs
duplicate_flights = Flights[Flights['Flight_ID'].duplicated()]

# Extract unique duplicate flight IDs
unique_duplicate_flight_ids = duplicate_flights['Flight_ID'].unique()

a = duplicate_flights.groupby('Flight_ID').agg({'STD': 'count'})

a.sort_values(by='STD', ascending=False)

,STD
Flight_ID,
63dca0dbfe6a982802a9fffdfc146049,3
b86769057b12484090d3a3e7f6435099,3
d036537b7fc6d99ded2fed7643c8b98a,3
13e310d6870b4b8473b4135bf79f4773,3
ecc73ab8941e49875e35235962e9c9ea,3
...,...
56a37cfe9bffcfea15c231a58aee4f17,1
5705b2916fef9e8f878c102e559036b7,1
5712bb2c8ee124f02fcb2a3651612434,1
